# Margin-implied price level approximation

## Price level approximation

Assumptions
  - Assume risk factors stay constant (which is the case for the log-normal model anyway).
  - Just focus on open position for now (ignore orders - these get cancelled upon closeout anyway and party's margin gets revaluated again to see if margin > maintenance in which case position doesn't get closed out).

Need to work out price `S` so that:

$$ \text{margin_level}(S)=\text{account_balance} + \text{P&L}(S) $$

where for futures:

$$ \text{margin_maintenance}(S)= max(|\text{open_volume}| * \text{slippage_per_unit}, 0) + |\text{open_volume}| * \text{risk_factor} * S $$
and

$$ \text{P&L}(S)= \text{open_volume}*[S-S_{\text{current}}] $$

### First approximation

Assumptions:
   - Assume margin levels don't get affected by price moves

As per assumptions above just calculate the P&L loss that'd bring margin account balance below **current** maintenance margin so we can approximate price levels associated with breaching search and maintenance margins as:

$$ S^*_\text{search} =  \frac{\text{margin_search}(S_\text{current}) - \text{margin_account_balance}}{\text{open_volume}} + S_\text{current}  $$

and 

$$ S^*_\text{liquidation} =  \frac{\text{margin_maintenance}(S_\text{current}) - \text{margin_account_balance}-\text{general_account_balance}}{\text{open_volume}} + S_\text{current}  $$


### Second approximation

Assumptions:
  - Assume slippage stays constant from when the position opens.

Let:

$$ c_1 = max(|\text{open_volume}| * \text{slippage_per_unit}, 0) $$
$$ c_2 = |\text{open_volume}| * \text{risk_factor} $$

and $c_{level}$ be the scaling for a particular risk level, hence we get:

$$S = \frac{\text{account_balance}-\text{open_volume}*S_\text{current}-c_{level}*c_1}{c_{level}*c_2-\text{open_volume}}$$

## Implementation

In [ ]:
import vegaapiclient as vac
from utils.risk_models import LogNormal
from utils.helpers import *

node_url_grpc="n06.testnet.vega.xyz:3007"
data_client = vac.VegaTradingDataClient(node_url_grpc)

### Specify public key for the party.
Specify the your public key in the cell below. It can be found in the wallet side panel in the Console.

In [ ]:
pubkey="73ce9e7fbda4e8ae4dec66e9786c17fd7a31a32a616b3287615b421877e3a8a6"

### Get party data

In [ ]:
party_positions = data_client.PositionsByParty(vac.data_node.api.v1.trading_data.PositionsByPartyRequest(party_id=pubkey))
if len(party_positions.positions) == 0:
    print("party with the public key specified has no open positions")
    quit(keep_kernel=True)
party_accounts = data_client.PartyAccounts(vac.data_node.api.v1.trading_data.PartyAccountsRequest(party_id=pubkey))

for pos in party_positions.positions:
    market=data_client.MarketByID(vac.data_node.api.v1.trading_data.MarketByIDRequest(market_id=pos.market_id)).market
    market_name=market.tradable_instrument.instrument.name
    market_data=data_client.MarketDataByID(vac.data_node.api.v1.trading_data.MarketDataByIDRequest(market_id=pos.market_id)).market_data
    margin_levels=data_client.MarginLevels(vac.data_node.api.v1.trading_data.MarginLevelsRequest(party_id=pubkey, market_id=pos.market_id)).margin_levels
  
    margin_acc = next(x for x in party_accounts.accounts if x.market_id == pos.market_id )
    gen_acc = next(x for x in party_accounts.accounts if x.type == vac.vega.vega.ACCOUNT_TYPE_GENERAL and x.asset==margin_acc.asset)
    
    asset_dp = data_client.AssetByID(vac.data_node.api.v1.trading_data.AssetByIDRequest(id=gen_acc.asset)).asset.details.decimals
    market_dp = market.decimal_places
    
    mar_acc_bal = float(margin_acc.balance)*10**(-asset_dp)
    gen_acc_bal = float(gen_acc.balance)*10**(-asset_dp)
    
    mar_mant = float(margin_levels[0].maintenance_margin)*10**(-market_dp)
    mar_srch = float(margin_levels[0].search_level)*10**(-market_dp)
    mar_init = float(margin_levels[0].initial_margin)*10**(-market_dp)
    mar_rel = float(margin_levels[0].collateral_release_level)*10**(-market_dp)
    
    mark_price=float(market_data.mark_price)*10**(-market_dp)
    

    
    search_price_first_approx = (mar_srch - mar_acc_bal)/pos.open_volume + mark_price
    liquidation_price_first_approx = (mar_mant - mar_acc_bal - gen_acc_bal)/pos.open_volume + mark_price

    p=market.tradable_instrument.log_normal_risk_model
    risk_model = LogNormal(mu=p.params.mu,sigma=p.params.sigma, lambd=p.risk_aversion_parameter, tau=p.tau)
    risk_factor = risk_model.RiskFactorLong() if pos.open_volume >= 0 else risk_model.RiskFactorShort()
    scaling_factors = market.tradable_instrument.margin_calculator.scaling_factors
    

    market_depth = data_client.MarketDepth(vac.data_node.api.v1.trading_data.MarketDepthRequest(market_id=market.id, max_depth=abs(pos.open_volume)))
    slippage_per_unit = calculate_slippage_per_unit(mark_price,market_depth, pos.open_volume, market_dp)


    c_1 = max(abs(pos.open_volume)*slippage_per_unit,0)
    c_2 = abs(pos.open_volume) * risk_factor

    search_price_second_approx = (mar_acc_bal -  pos.open_volume*mark_price-scaling_factors.search_level *c_1 )/(scaling_factors.search_level*c_2-pos.open_volume)
    liquidation_price_second_approx = (mar_acc_bal + gen_acc_bal - pos.open_volume*mark_price-c_1 )/(c_2-pos.open_volume)

    
    print("\tmarket: {name}".format(name=market_name)) 
    print("\n")
    print("\t\t{:<29} {}".format("party position:",pos.open_volume))
    print("\t\t{:<29} {}".format("margin account:",round_sd(mar_acc_bal)))
    print("\t\t{:<29} {}".format("general account:",round_sd(gen_acc_bal)))
    print("\t\t{:<29} {}".format("mark price:",round_sd(mark_price)))
    print("\n")
    print("\t\tmargin levels:")
    print("\t\t\t{:<20} ~{}".format("search:",round_sd(mar_srch)))
    print("\t\t\t{:<20} ~{}".format("maintenance:",round_sd(mar_mant)))
    print("\n")
    print("\t\tfirst approximation price:")
    print("\t\t\t{:<20} ~{}".format("search:",max(0,round_sd(search_price_first_approx))))
    print("\t\t\t{:<20} ~{}".format("liquidation:",max(0,round_sd(liquidation_price_first_approx))))
    print("\t\tsecond approximation price:")
    print("\t\t\t{:<20} ~{}".format("search:",max(0,round_sd(search_price_second_approx))))
    print("\t\t\t{:<20} ~{}".format("liquidation:",max(0,round_sd(liquidation_price_second_approx))))
    print("\n\n")
    

## Appendix: Maintenance margin calculation

```maintenance_margin = max ( maintenance_margin_long, maintenance_margin_short)```

riskiest long = max( open_volume + buy_orders , 0 )

riskiest short = min( open_volume + sell_orders, 0 )

### `maintenance_margin_long`
(calculation for `maintenance_margin_short` is equivalent, see [spec](https://github.com/vegaprotocol/specs-internal/blob/master/protocol/0019-margin-calculator.md) for details)

```maintenance_margin_long_open_position = max(slippage_volume * slippage_per_unit, 0) + slippage_volume * [ quantitative_model.risk_factors_long ] . [ Product.value(market_observable) ]```,

```maintenance_margin_long_open_orders = buy_orders * [ quantitative_model.risk_factors_long ] . [ Product.value(market_observable) ]  ```,

where

```slippage_volume =  max( open_volume, 0 ) ```,

and

if ```open_volume > 0```  then 

```slippage_per_unit =  Product.value(market_observable) - Product.value(exit_price) ```, 

else ```slippage_per_unit = 0```.